In [1]:
import torch, spacy
from nltk.corpus import wordnet
from transformers import T5Tokenizer, T5ForConditionalGeneration
import re

torch.cuda.empty_cache()

nlp = spacy.load("en_core_web_sm")

/home/arkin/python-venvs/bert-counterfactual/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the fine-tuned model
model_path = "./t5_finetuned_airline_incidents"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def predict_failure(incident_description):
    input_text = f"Report: {incident_description}\nPart Failure: "
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

    # Generate Prediction
    with torch.no_grad():
        output = model.generate(**inputs, max_length=128)

    predicted_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return predicted_text

In [4]:
# Replace Causal Words with Synonyms

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace("_", " ")  # Replace underscores with spaces
            if synonym.lower() != word.lower():  # Exclude the original word
                synonyms.add(synonym)
    return list(synonyms)

def get_one_synonym(word):
    for syn in wordnet.synsets(word):  # Get synsets for the word
        for lemma in syn.lemmas():  # Get lemmas within synset
            if lemma.name() != word:
                print(f"'{word}' --> '{lemma.name()}'")
                return lemma.name()  # Return the first found synonym
    
    print(f"'{word}' -->")
    return word  # Return None if no synonym is found

def replace_word_with_synonym(text: str, word: str):
    modified_text = text
    
    synonym = get_one_synonym(word)
    
    pattern = re.compile(word, re.IGNORECASE)
    modified_text = pattern.sub(synonym, text)
    
    return modified_text, synonym

In [5]:
REPORT = "THE FLIGHT CREW REPORTED OF A BAGGAGE/FUEL DOOR CAS MESSAGE. OPERATIONS WERE CONTINUED PER MEL 52-4. THE MAINTENANCE TEAM TROUBLESHOT AND FOUND THE FUEL DOOR MICROSWITCH TO BE DEFECTIVE. THE SWITCH WAS REPLACED WITH NEW AND RIGGED. OPERATIONS WERE SATISFACTORY AND THE AIRCRAFT WAS RETURNED TO SERVICE."
print(f"Original Report: {REPORT}")

origial_response = predict_failure(REPORT)
print(f"Original Response: {origial_response}\n")

report_without_special_characters = re.sub(r'[^A-Za-z ]', '', REPORT)
report_word_list = report_without_special_characters.lower().split(" ")

causal_word = []
alternative_reponses = {}
sub_num = 0

for word in report_word_list:

    word_affected_result = False
    
    synonyms_list = get_synonyms(word)
    for synonym in synonyms_list:

        pattern = re.compile(word, re.IGNORECASE)
        modified_report = pattern.sub(synonym, REPORT)
        
        new_response = predict_failure(modified_report)
        
        sub_num += 1

        print(f"\n'{word}' --> '{synonym}'")
        if new_response != origial_response:
            word_affected_result = True
            print(f"----> Observed Change <----")
            print(f"New Report: {modified_report}")
            print(f"Changed Response: '{origial_response}' --> '{new_response}'")
            print("")
            alternative_reponses[f"'{word}' --> '{synonym}'"] = new_response
        else:
            print(f"No Observed Change.")

    if word_affected_result:
        causal_word.append(word)

print("\n--------------- Summary ---------------")
print(f"Original Report: '{REPORT}'")
print(f"Original Response: '{origial_response}'")
print(f"Number of substitutions: {sub_num}")
print(f"Number of observed changes: {len(alternative_reponses)}")

print("\nCausal Words:")
for i in causal_word:
    print(f"'{i}'")

print("\nAlternative Responses:")
for i in alternative_reponses:
    print(f"{i} generates the response: '{alternative_reponses[i]}'")



Original Report: THE FLIGHT CREW REPORTED OF A BAGGAGE/FUEL DOOR CAS MESSAGE. OPERATIONS WERE CONTINUED PER MEL 52-4. THE MAINTENANCE TEAM TROUBLESHOT AND FOUND THE FUEL DOOR MICROSWITCH TO BE DEFECTIVE. THE SWITCH WAS REPLACED WITH NEW AND RIGGED. OPERATIONS WERE SATISFACTORY AND THE AIRCRAFT WAS RETURNED TO SERVICE.


/home/arkin/python-venvs/bert-counterfactual/lib64/python3.11/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:310.)
  return F.linear(input, self.weight, self.bias)


Original Response: failure: FUEL DOOR DEFECTIVE


'flight' --> 'trajectory'
No Observed Change.

'flight' --> 'flying'
No Observed Change.

'flight' --> 'fledge'
No Observed Change.

'flight' --> 'flight of stairs'
No Observed Change.

'flight' --> 'flight of steps'
No Observed Change.

'flight' --> 'escape'
No Observed Change.

'crew' --> 'gang'
No Observed Change.

'crew' --> 'bunch'
No Observed Change.

'crew' --> 'work party'
No Observed Change.

'crew' --> 'crowd'
No Observed Change.

'reported' --> 'report'
No Observed Change.

'reported' --> 'cover'
No Observed Change.

'reported' --> 'describe'
No Observed Change.

'reported' --> 'account'
No Observed Change.

'a' --> 'axerophthol'
No Observed Change.

'a' --> 'adenine'
No Observed Change.

'a' --> 'angstrom'
No Observed Change.

'a' --> 'group A'
No Observed Change.

'a' --> 'angstrom unit'
No Observed Change.

'a' --> 'antiophthalmic factor'
No Observed Change.

'a' --> 'amp'
No Observed Change.

'a' --> 'deoxyadenosine monop